In [1]:
import sqlite3

# 새로운 메모리 기반 DB 생성
conn = sqlite3.connect(":memory:", check_same_thread=False)
cursor = conn.cursor()
cursor.execute("CREATE TABLE inventory(id INTEGER PRIMARY KEY, stock INTEGER)")
cursor.execute("INSERT INTO inventory(id, stock) VALUES(1,100)")
conn.commit()

In [2]:
import threading
import time


def shared_read():
    local_conn = sqlite3.connect(":memory:", check_same_thread=False)
    c = local_conn.cursor()
    c.execute("SELECT stock FROM inventory WHERE id=1")
    print("공유락(Shared Lock): 재고 =", c.fetchone())
    local_conn.close()


def exclusive_write():
    local_conn = sqlite3.connect(":memory:", check_same_thread=False)
    c = local_conn.cursor()
    c.execute("BEGIN EXCLUSIVE")
    c.execute("UPDATE inventory SET stock=stock-1 WHERE id=1")
    print("배타락(Exclusive Lock): 재고 차감 완료")
    time.sleep(5)  # 락 점유 시간 연장
    local_conn.commit()
    local_conn.close()


t1 = threading.Thread(target=exclusive_write)
t2 = threading.Thread(target=shared_read)

t1.start()
time.sleep(1)  # 배타락 우선 확보
t2.start()
t1.join()
t2.join()


Exception in thread Thread-5 (exclusive_write):
Traceback (most recent call last):
  File "C:\Users\hy\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "d:\learned\TIL\.venv\Lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "C:\Users\hy\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\hy\AppData\Local\Temp\ipykernel_45848\2369922297.py", line 17, in exclusive_write
sqlite3.OperationalError: no such table: inventory
Exception in thread Thread-6 (shared_read):
Traceback (most recent call last):
  File "C:\Users\hy\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "d:\learned\TIL\.venv\Lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "C:\Users\hy\AppData\Local\Progr